In [16]:
import kss
import soynlp
from pykospacing import Spacing
from hanspell import spell_checker

import pandas as pd
import re

In [157]:
train_dir = './preprocessed_NNG.csv'
train_df = pd.read_csv(train_dir)
train_df = train_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
train_df.head(10)

,word,type,v1,v2,v3,v4
0,새물,NNG,1.새로 갓 나온 과일이나 생선 따위를 이르는 말.2.빨래하여 이제 막 입은 옷.[...,새로 갓 나온 과일이나 생선 따위를 이르는 말,새로 갓 나온 과일이나 생선 따위를 이르는 말,새로 갓 나온 과일이나 생선 따위를 이르는 말
1,합병증,NNG,[의학 ]어떤 질병에 곁들여 일어나는 다른 질병.,어떤 질병에 곁들여 일어나는 다른 질병,어떤 질병에 곁들여 일어나는 다른 질병,어떤 질병에 곁들여 일어나는 다른 질병
2,머름,NNG,[건설 ]바람을 막거나 모양을 내기 위하여 미닫이 문지방 아래나 벽 아래 중방에 대...,바람을 막거나 모양을 내기 위하여 미닫이 문지방 아래나 벽 아래 중방에 대는 널조각,바람을 막거나 모양을 내기 위하여 미닫이 문지방 아래나 벽 아래 중방에 대는 널조각,바람을 막거나 모양을 내기 위하여 미닫이 문지방 아래나 벽 아래 중방에 대는 널조각
3,그래픽,NNG,"그림이나 도형, 사진 등 다양한 시각적 형상(形象)이나 작품을 통틀어 이르는 말.","그림이나 도형, 사진 등 다양한 시각적 형상(形象)이나 작품을 통틀어 이르는 말.","그림이나 도형, 사진 등 다양한 시각적 형상(形象)이나 작품을 통틀어 이르는 말.","그림이나 도형, 사진 등 다양한 시각적 형상(形象)이나 작품을 통틀어 이르는 말."
4,자기력선,NNG,[물리 ]자기장의 크기와 방향을 나타내는 선. 유리판 위에 쇳가루를 뿌리고 그 유리...,자기장의 크기와 방향을 나타내는 선,자기장의 크기와 방향을 나타내는 선,자기장의 크기와 방향을 나타내는 선
5,분패,NNG,"일을 잡쳐서 실패함.[유의어]낭패,망단3,실패2","일을 잡쳐서 실패함.[유의어]낭패,망단3,실패2",일을 잡쳐서 실패함.,일을 잡쳐서 실패함.
6,반도라,NNG,"[음악 ]기타와 비슷하며 낮은 소리를 내는, 르네상스 시대의 유럽 민속 현악기.","기타와 비슷하며 낮은 소리를 내는, 르네상스 시대의 유럽 민속 현악기","기타와 비슷하며 낮은 소리를 내는, 르네상스 시대의 유럽 민속 현악기","기타와 비슷하며 낮은 소리를 내는, 르네상스 시대의 유럽 민속 현악기"
7,벨벳,NNG,"거죽에 곱고 짧은 털이 촘촘히 돋게 짠 비단.[유의어]비로드,우단3","거죽에 곱고 짧은 털이 촘촘히 돋게 짠 비단.[유의어]비로드,우단3",거죽에 곱고 짧은 털이 촘촘히 돋게 짠 비단.,거죽에 곱고 짧은 털이 촘촘히 돋게 짠 비단.
8,펌프,NNG,"1.수도 시설이 없는 곳에서, 사람이 손잡이를 상하로 되풀이하여 움직임으로써 그 압...",Longer,Longer,Longer
9,세균,NNG,[생명 ]생물체 가운데 가장 미세하고 가장 하등에 속하는 단세포 생활체. 다른 생물...,생물체 가운데 가장 미세하고 가장 하등에 속하는 단세포 생활체,생물체 가운데 가장 미세하고 가장 하등에 속하는 단세포 생활체,생물체 가운데 가장 미세하고 가장 하등에 속하는 단세포 생활체


In [77]:
sample_context = list(train_df.tail(20)['v4'])
sample_context

['다른 물건이 닿거나 묻어서 생긴 자리',
 '사물이나 일 따위의 기본이 되는 것',
 '영국의 성직자(1573~1645)',
 '형의 선고를 받은 사람의 형벌을 줄여 주는 일',
 '어떤 파동 또는 물리적 작용을 한 곳에서 다른 곳으로 옮겨 주는 매개물',
 '세력이나 힘을 더 강하고 튼튼하게 함',
 '어떤 사람이나 존재를 몹시 아끼고 귀중히 여기는 마음',
 '고려 시대에, 벼슬아치나 공신(功臣) 또는 각 관아에 토지 및 땔나무를 댈 임야를 나누어 주던 제도',
 '지구의 표면. 또는 땅의 겉면.',
 '익은 농작물을 거두어들임',
 '하나를 반으로 가름',
 '사람이나 동물이 먹은 음식물을 소화하여 항문으로 내보내는 찌꺼기',
 '분비샘에서 나오는 물질',
 '우주에 존재하는 모든 물체',
 '어떤 사실의 앞뒤, 또는 두 사실이 이치상 어긋나서 서로 맞지 않음을 이르는 말',
 '[역사 ]고려 시대에, 과거(科擧) 삼공(三貢)의 하나',
 '열복사로 방출된 전자기파가 물체에 흡수되어 그 물체를 뜨겁게 하는 에너지',
 'Longer',
 '필요한 물자를 스스로 생산하여 충당함.',
 '나비목의 곤충 가운데 낮에 활동하는 무리를 통틀어 이르는 말']

In [78]:
def remove_useless_breacket(texts):
    """
    위키피디아 전처리를 위한 함수입니다.
    괄호 내부에 의미가 없는 정보를 제거합니다.
    아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
    ``수학(,)`` -> ``수학``
    ``수학(數學,) -> ``수학(數學)``
    """
    bracket_pattern = re.compile(r"\((.*?)\)")
    preprocessed_text = []
    for text in texts:
        modi_text = ""
        text = text.replace("()", "")  # 수학() -> 수학
        brackets = bracket_pattern.search(text)
        if not brackets:
            if text:
                preprocessed_text.append(text)
                continue
        replace_brackets = {}
        # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
        # e.g. {'2,8': '(數學)','34,37': ''}
        while brackets:
            index_key = str(brackets.start()) + "," + str(brackets.end())
            bracket = text[brackets.start() + 1 : brackets.end() - 1]
            infos = bracket.split(",")
            modi_infos = []
            for info in infos:
                info = info.strip()
                if len(info) > 0:
                    modi_infos.append(info)
            if len(modi_infos) > 0:
                replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
            else:
                replace_brackets[index_key] = ""
            brackets = bracket_pattern.search(text, brackets.start() + 1)
        end_index = 0
        for index_key in replace_brackets.keys():
            start_index = int(index_key.split(",")[0])
            modi_text += text[end_index:start_index]
            modi_text += replace_brackets[index_key]
            end_index = int(index_key.split(",")[1])
        modi_text += text[end_index:]
        modi_text = modi_text.strip()
        if modi_text:
            preprocessed_text.append(modi_text)
    return preprocessed_text

In [79]:
def clean_punc(texts):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

    preprocessed_text = []
    for text in texts:
        for p in punct_mapping:
            text = text.replace(p, punct_mapping[p])
        text = text.strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [81]:
def remove_repeated_spacing(texts):
    """
    두 개 이상의 연속된 공백을 하나로 치환합니다.
    ``오늘은    날씨가   좋다.`` -> ``오늘은 날씨가 좋다.``
    """
    preprocessed_text = []
    for text in texts:
        text = re.sub(r"\s+", " ", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [82]:
def spacing_sent(texts):
    spacing = Spacing()
    """
    띄어쓰기를 보정합니다.
    """
    preprocessed_text = []
    for text in texts:
        text = spacing(text)
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [83]:
def spell_check_sent(texts):
    """
    맞춤법을 보정합니다.
    """
    preprocessed_text = []
    for text in texts:
        try:
            spelled_sent = spell_checker.check(text)
            checked_sent = spelled_sent.checked 
            if checked_sent:
                preprocessed_text.append(checked_sent)
        except:
            preprocessed_text.append(text)
    return preprocessed_text

In [84]:
def remove_stopwords(sents, stop_word_list):
    #  큰 의미가 없는 불용어 정의
    stopwords = stop_word_list
    preprocessed_text = []
    for sent in sents:
        sent = [w for w in sent.split(' ') if w not in stopwords]# 불용어 제거
        preprocessed_text.append(' '.join(sent))
    return preprocessed_text

In [85]:
def min_max_filter(texts,min_len = 2, max_len = 500):
    """
    문장을 최대, 최소 길이로 필터링합니다.
    """
    preprocessed_text = []
    for text in texts:
        preprocessed_text.append(re.sub(r'\(.*\)', '', text))
    return preprocessed_text

In [86]:
def remove_additional_brackets(texts):
    """
    sy; [ ], ( ) 사이값들 다 지워버리기
    """
    # text = '[역사 ]고려 시대에, 과거(科擧) 삼공(三貢)의 하나'
    # out = re.sub(r'\[.*\]', '', text)
    # print(out)
    
    preprocessed_text = []
    for text in texts:
        out = re.sub(r'\[.*\]', '', text)
        start_bracket = list(re.finditer('\(', out))
        end_bracket = list(re.finditer('\)', out))
        new_str = ''
        anchor = 0
        for _, (starts, ends) in enumerate(zip(start_bracket, end_bracket)):
            start_idx = starts.start()
            end_idx = ends.end()
            new_str += out[anchor:start_idx]
            anchor = end_idx
        new_str += out[anchor:]
        preprocessed_text.append(new_str)
    return preprocessed_text

In [87]:
# remove_additional_brackets(sample_context)

In [88]:
def remove_hanja(texts):
    preprocessed_text = []
    pattern = re.compile(r'[,.^ ㄱ-ㅣ가-힣]+') #'[^ ㄱ-ㅣ가-힣]+' , '[ㄱ-ㅣ가-힣]'
    # result = hangul.sub('', s)
    # results = pattern.sub('', string)
    for text in texts:
        results = pattern.findall(text)
        if len(results) > 1:
            preprocessed_text.append('not hangeul only')
        else:
            preprocessed_text.append(text)
    return preprocessed_text

In [40]:
string = '그림이나 도형, 사진 등 다양한 시각적 형상이나 작품을 통틀어 이르는 말.'
remove_hanja(string)

['그림이나 도형, 사진 등 다양한 시각적 형상이나 작품을 통틀어 이르는 말.']


'그림이나 도형, 사진 등 다양한 시각적 형상이나 작품을 통틀어 이르는 말.'

In [89]:
def run_preprocess(context):

    #불필요한 괄호제거
    context = remove_useless_breacket(context)
    #문장부호제거
    context = clean_punc(context)
    #연속공백제거
    context = remove_repeated_spacing(context)
    #연속 띄어쓰기 제거
    #context = spacing_sent(context)
    #불용어 제거
    stop_word_list = ['']
    context = remove_stopwords(context, stop_word_list)
    #최소최대길이 제한
    context = min_max_filter(context)

    # [] 제거, 여러개 ( * ) 제거
    context = remove_additional_brackets(context)
    
    # 위에 제거했는데도 한자 있는 애들은 제외 시키기
    context = remove_hanja(context)
    preprocessed_context = context
    return preprocessed_context

In [43]:
# https://kynk94.github.io/devlog/post/re-match-hangul
# https://jokergt.tistory.com/52
import re
string = "한자 부수의 하나. '鼾', '齈' 따위에 쓰인 '鼻'를 이른다."
pattern = re.compile(r'[^ ㄱ-ㅣ가-힣]+') #'[^ ㄱ-ㅣ가-힣]+' , '[ㄱ-ㅣ가-힣]'
# result = hangul.sub('', s)
# results = pattern.sub('', string)
results = pattern.findall(string)
# if len(results) > 1:

In [154]:
sample_context = list(train_df['v4'])

In [158]:
pro = run_preprocess(sample_context)
train_df.insert(column='v5', value=pro, loc=len(train_df.columns))
train_df=train_df[train_df['v5']!='not hangeul only']
# for idx, (senta, sentb) in enumerate(zip(sample_context, pro)):
#     print(idx, senta, '\t', sentb)

In [159]:
train_df.to_csv('preprocessed_NNG_v5.csv')
# train_df

In [142]:
train_dir = './preprocessed_VA.csv'
train_df = pd.read_csv(train_dir)
train_df = train_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
# train_df.head(10)

In [143]:
sample_context = list(train_df['v4'])
pro = run_preprocess(sample_context)
train_df.insert(column='v5', value=pro, loc=len(train_df.columns))

In [144]:
add_train_dir = './WAI-para_VA_antonyms.csv'
add_train_df = pd.read_csv(add_train_dir)
VA_ant_df = add_train_df[add_train_df.is_replace==1].drop(columns='Unnamed: 0')

In [145]:
mapping = {VA_ant_df['VA'].iloc[idx]: VA_ant_df['antonyms'].iloc[idx] for idx in range(len(VA_ant_df))}

In [146]:
ant = []
for _, (word, type_, _, _,_,_, _, _) in train_df.iterrows():
    if word in mapping:
        ant.append(mapping[word])
    else:
        ant.append('NaN')

In [147]:
train_df.insert(column='반의어', value=ant, loc=len(train_df.columns))
train_df

,word,type,v1,v2,v3,유의어,v4,v5,반의어
0,즐겁다,VA,"마음에 거슬림이 없이 흐뭇하고 기쁘다.[유의어]고소하다1,기쁘다,달다7","마음에 거슬림이 없이 흐뭇하고 기쁘다.[유의어]고소하다1,기쁘다,달다7",마음에 거슬림이 없이 흐뭇하고 기쁘다.,"['고소하다', '기쁘다', '달다']",마음에 거슬림이 없이 흐뭇하고 기쁘다.,마음에 거슬림이 없이 흐뭇하고 기쁘다.,지루하다
1,갓다,VA,Not verb,Not verb,Not verb,NaN,Not verb,Not verb,NaN
2,벅차다,VA,"1.감당하기가 어렵다.2.감격, 기쁨, 희망 따위가 넘칠 듯이 가득하다.3.숨이 견...",감당하기가 어렵다,감당하기가 어렵다,NaN,감당하기가 어렵다,감당하기가 어렵다,NaN
3,어뜨다,VA,Not verb,Not verb,Not verb,NaN,Not verb,Not verb,NaN
4,낡다,VA,"1.물건 따위가 오래되어 헐고 너절하게 되다.2.생각이나 제도, 문물 따위가 시대에...",물건 따위가 오래되어 헐고 너절하게 되다,물건 따위가 오래되어 헐고 너절하게 되다,NaN,물건 따위가 오래되어 헐고 너절하게 되다,물건 따위가 오래되어 헐고 너절하게 되다,새롭다
...,...,...,...,...,...,...,...,...,...
107,읎다,VA,"<방언>‘없다’의 방언 (강원, 경북, 전남, 충청)","<방언>‘없다’의 방언 (강원, 경북, 전남, 충청)","<방언>‘없다’의 방언 (강원, 경북, 전남, 충청)",NaN,"<방언>‘없다’의 방언 (강원, 경북, 전남, 충청)",not hangeul only,NaN
108,배고프다,VA,1.배 속이 비어서 음식이 먹고 싶다.2.끼니를 잇지 못할 정도로 생활이 넉넉하지 ...,배 속이 비어서 음식이 먹고 싶다,배 속이 비어서 음식이 먹고 싶다,NaN,배 속이 비어서 음식이 먹고 싶다,배 속이 비어서 음식이 먹고 싶다,배부르다
109,수많다,VA,"수효가 매우 많다.[유의어]다수하다,많다,삼라하다","수효가 매우 많다.[유의어]다수하다,많다,삼라하다",수효가 매우 많다.,"['다수하다', '많다', '삼라하다']",수효가 매우 많다.,수효가 매우 많다.,적다
110,놉다,VA,[방언 ]‘노엽다’의 방언(함북).,‘노엽다’의 방언(함북),‘노엽다’의 방언(함북),NaN,‘노엽다’의 방언(함북),not hangeul only,NaN


In [150]:
train_df = train_df[train_df.v5!='Not verb']
train_df = train_df[train_df.v5!='not hangeul only']
train_df = train_df[train_df.v5!='Start with special character']

In [151]:
train_df

,word,type,v1,v2,v3,유의어,v4,v5,반의어
0,즐겁다,VA,"마음에 거슬림이 없이 흐뭇하고 기쁘다.[유의어]고소하다1,기쁘다,달다7","마음에 거슬림이 없이 흐뭇하고 기쁘다.[유의어]고소하다1,기쁘다,달다7",마음에 거슬림이 없이 흐뭇하고 기쁘다.,"['고소하다', '기쁘다', '달다']",마음에 거슬림이 없이 흐뭇하고 기쁘다.,마음에 거슬림이 없이 흐뭇하고 기쁘다.,지루하다
2,벅차다,VA,"1.감당하기가 어렵다.2.감격, 기쁨, 희망 따위가 넘칠 듯이 가득하다.3.숨이 견...",감당하기가 어렵다,감당하기가 어렵다,NaN,감당하기가 어렵다,감당하기가 어렵다,NaN
4,낡다,VA,"1.물건 따위가 오래되어 헐고 너절하게 되다.2.생각이나 제도, 문물 따위가 시대에...",물건 따위가 오래되어 헐고 너절하게 되다,물건 따위가 오래되어 헐고 너절하게 되다,NaN,물건 따위가 오래되어 헐고 너절하게 되다,물건 따위가 오래되어 헐고 너절하게 되다,새롭다
5,가쁘다,VA,"1.숨이 몹시 차다.2.힘에 겹다.3.몹시 급하거나 빠르다.[유의어]고되다,급하다,...",숨이 몹시 차다,숨이 몹시 차다,NaN,숨이 몹시 차다,숨이 몹시 차다,여유롭다
6,다름없다,VA,"견주어 보아 같거나 비슷하다.[유의어]같다,동일하다,진배없다","견주어 보아 같거나 비슷하다.[유의어]같다,동일하다,진배없다",견주어 보아 같거나 비슷하다.,"['같다', '동일하다', '진배없다']",견주어 보아 같거나 비슷하다.,견주어 보아 같거나 비슷하다.,다르다
...,...,...,...,...,...,...,...,...,...
103,멋있다,VA,"보기에 썩 좋거나 훌륭하다.[유의어]근사하다1,멋거리지다,멋들어지다","보기에 썩 좋거나 훌륭하다.[유의어]근사하다1,멋거리지다,멋들어지다",보기에 썩 좋거나 훌륭하다.,"['근사하다', '멋거리지다', '멋들어지다']",보기에 썩 좋거나 훌륭하다.,보기에 썩 좋거나 훌륭하다.,NaN
105,거칠다,VA,1.나무나 살결 따위가 결이 곱지 않고 험하다.2.피륙의 올이 성기고 굵다.3.일을...,나무나 살결 따위가 결이 곱지 않고 험하다,나무나 살결 따위가 결이 곱지 않고 험하다,NaN,나무나 살결 따위가 결이 곱지 않고 험하다,나무나 살결 따위가 결이 곱지 않고 험하다,부드럽다
106,게으르다,VA,"행동이 느리고 움직이거나 일하기를 싫어하는 성미나 버릇이 있다.[유의어]개르다,개으...","행동이 느리고 움직이거나 일하기를 싫어하는 성미나 버릇이 있다.[유의어]개르다,개으...",행동이 느리고 움직이거나 일하기를 싫어하는 성미나 버릇이 있다.,"['개르다', '개으르다', '나만하다']",행동이 느리고 움직이거나 일하기를 싫어하는 성미나 버릇이 있다.,행동이 느리고 움직이거나 일하기를 싫어하는 성미나 버릇이 있다.,성실하다
108,배고프다,VA,1.배 속이 비어서 음식이 먹고 싶다.2.끼니를 잇지 못할 정도로 생활이 넉넉하지 ...,배 속이 비어서 음식이 먹고 싶다,배 속이 비어서 음식이 먹고 싶다,NaN,배 속이 비어서 음식이 먹고 싶다,배 속이 비어서 음식이 먹고 싶다,배부르다


In [152]:
train_df.to_csv('preprocessed_VA_v5.csv')